In [7]:
import nasdaqdatalink



In [3]:
data=nasdaqdatalink.bulkdownload("ZEA",nasdaq_key)

ForbiddenError: (Status 401) (Nasdaq Data Link Error QEPx01) You have attempted to view a protected resource in anonymous mode, i.e., without providing a Nasdaq Data Link key. Please register for a free Nasdaq Data Link account, and then include your API key with your requests.